In [1]:
import scipy.stats as st
import simpy
import numpy as np



In [2]:
import scipy.stats as st
import simpy
import numpy as np
from queue import Queue
import pandas as pd
    

     

class sensor ():
    def __init__(self,type,arrival_rate,data_size,priority_level,avg_processing,fixed_size=True,fixed_arrival=True) :
        self.name=type
        self.arrival_rate = arrival_rate
        self.fixed_arrival =fixed_arrival
        self.data_size=data_size 
        self.fixed_size =fixed_size
        self.priority_level=priority_level
        self.avg_processing=avg_processing
        
    def get_arrival_rate(self):
            return self.arrival_rate if self.fixed_arrival ==True else  self.arrival_rate.rvs()
    def get_data_size(self):
        return self.data_size if self.fixed_size ==True else  self.data_size.rvs()
    def get_processing_time(self):
        return st.expon(0,self.avg_processing).rvs()
    
class Sensor_hold():
     def __init__(self,name,arrival_time,data_size,priority_level,processing) :
          self.name = name
          self.arrival_time=arrival_time
          self.data_size=data_size
          self.priority_level= priority_level
          self.process_time=processing
    
    

In [3]:
class HomeIot():
    def __init__(self,cores=['core 1','core 2','core 3','core 4'],capacity=128*10**6):
        self.nb_rejected=0
        self.queue = []
        self.cores_usage={
            'core 1':0,
            'core 2':0,
            'core 3':0,
            'core 4':0
        }
        self.capacity = capacity
        self.cores =cores
        self.sensors=[]
        self.data={'name':[],
                   'arrival_time':[],
                   'begin_time':[],
                   'ending_time':[],
                   'waiting_time':[],
                   'priority':[],
                   'data_size(Ko)':[],
                   'process_time':[],
                   'cores':[],
                   
                    }

        self.dataset= {
            'nb_arrivals': 0,
            'nb_rejected': 0,
            'sum_process_time':0,
            'waiting_time':[]
        }
        self.env = simpy.Environment()
        self.ressources = simpy.Resource(self.env,len(self.cores))
        sensor1= sensor('IntrusionDetector',30,st.norm(128*(10**3),32*(10**3)),4,15,fixed_size=False)
        self.generator = self.env.process(self.sensors_arrival(sensor1))

        sensor2= sensor('GasDetector',60,10**3,4,5)
        self.generator = self.env.process(self.sensors_arrival(sensor2))

        sensor3= sensor('VocalDetector',st.expon(0,3600/12),256*10**3,3,10,fixed_arrival=False)
        self.generator = self.env.process(self.sensors_arrival(sensor3))

        sensor4= sensor('Camera',10,st.norm(2*(10**6),512*(10**3)),2,15,fixed_size=False)
        self.generator = self.env.process(self.sensors_arrival(sensor4))

        sensor5= sensor('TempDetector',180,10**3,1,5)
        self.generator = self.env.process(self.sensors_arrival(sensor5))
        

    def compute_waiting_time_for_sensor(self, sensor_name):
        data = pd.DataFrame(self.data)
        sensor_data = data[data['name'] == sensor_name]
        
        return list(sensor_data['waiting_time'])
    
    def sensors_arrival(self,sensor):
        # print('run')    

        while True :

            self.dataset['nb_arrivals'] +=1
            yield self.env.timeout(sensor.get_arrival_rate())
            self.arrival_time = self._clock()
            self.data_size = sensor.get_data_size()
            self.sensor_hold = Sensor_hold(sensor.name,self.arrival_time,self.data_size,sensor.priority_level,sensor.get_processing_time())         
            self.sensors.append(sensor)
            if (self.capacity - self.sensor_hold.data_size) > 0  :
                self.queue.append(self.sensor_hold)

                self.capacity-=self.sensor_hold.data_size
                
            else:
                self.dataset['nb_rejected'] +=1
            self.env.process(self.process_sensors())
        

        

    def process_sensors(self):
        arrival_time = self._clock()
        with self.ressources.request() as req:
            yield req

            if not self.queue:
                # The queue is empty, nothing to process
                return

            sensor_index = self.get_sensor()
            sensor = self.queue.pop(sensor_index)
            begin_time = self._clock()            
            waiting_time = begin_time - sensor.arrival_time
            ending_time = begin_time + sensor.process_time
            self.data['name'].append(sensor.name)
            self.data['arrival_time'].append(sensor.arrival_time)
            self.data['begin_time'].append(begin_time)
            self.data['ending_time'].append(ending_time)
            self.data['waiting_time'].append(waiting_time)
            self.data['priority'].append(sensor.priority_level)
            self.data['data_size(Ko)'].append(sensor.data_size/10**3)
            self.data['process_time'].append(sensor.process_time)

            self.capacity += sensor.data_size
            self.dataset['waiting_time'].append(waiting_time)
            core = self.cores.pop(0)
            core_begin= self._clock()         
            self.data['cores'].append(core) 
            yield self.env.timeout(sensor.process_time)
            self.cores.append(core)
            core_end= self._clock()         
            self.cores_usage[core]+=core_end - core_begin

    

    # Get the instance with the highest priority level
    def get_sensor(self):
        k=0
       
        for index,sensor in enumerate(self.queue):
            if sensor.priority_level> self.queue[k].priority_level:
                k= index
        
        return k
        
    def _clock(self): 
        return self.env.now
    
    def run(self, stop_time=100, print_details=False):   
        self.env.run(until=stop_time)  # launch the simulation for a given duration
        self.stats()
        
    def stats(self):
        df = self.dataset
        df['mean_proc']=df['sum_process_time']/ (df['nb_arrivals']-df['nb_rejected'])
        df['percentage_rejected']=df['nb_rejected']*100/df['nb_arrivals']







# 1)
### Simulate this smart home IoT system queue for 24 hours without displaying details and compute/display the following queue statistics:

In [4]:
home = HomeIot()
stop_time = 24*3600
home.run(stop_time)


In [5]:
df= home.dataset
data=pd.DataFrame(home.data)
print(df['nb_arrivals'])
print(df['nb_rejected'])
print(f'percentage rejected :{df["percentage_rejected"]}%')

print(f'mean of waiting time = {(np.mean(df["waiting_time"]))}')
print(f'max of waiting time = {np.max(df["waiting_time"])}')
print(f'mean processing time  :{np.mean(data["process_time"])}')

print(f'core1 usage : {(home.cores_usage["core 1"]*100/stop_time):.2f}% of the time')
print(f'core2 usage : {(home.cores_usage["core 2"]*100/stop_time):.2f}% of the time')
print(f'core3 usage : {(home.cores_usage["core 3"]*100/stop_time):.2f}% of the time')
print(f'core4 usage : {(home.cores_usage["core 4"]*100/stop_time):.2f}% of the time')
data.head(len(data))

13731
0
percentage rejected :0.0%
mean of waiting time = 0.4950152687027064
max of waiting time = 44.438538142087054
mean processing time  :13.535176096086497
core1 usage : 53.31% of the time
core2 usage : 54.24% of the time
core3 usage : 53.62% of the time
core4 usage : 53.81% of the time


,name,arrival_time,begin_time,ending_time,waiting_time,priority,data_size(Ko),process_time,cores
0,Camera,10.0,10.0,15.391711,0.0,2,1484.265770,5.391711,core 1
1,Camera,20.0,20.0,48.856541,0.0,2,2864.490278,28.856541,core 2
2,IntrusionDetector,30.0,30.0,39.879245,0.0,4,84.739832,9.879245,core 3
3,Camera,30.0,30.0,58.263863,0.0,2,2842.993112,28.263863,core 4
4,Camera,40.0,40.0,71.536691,0.0,2,1621.261185,31.536691,core 1
...,...,...,...,...,...,...,...,...,...
13721,Camera,86360.0,86360.0,86370.353059,0.0,2,1402.071943,10.353059,core 4
13722,IntrusionDetector,86370.0,86370.0,86376.590502,0.0,4,34.237223,6.590502,core 3
13723,Camera,86370.0,86370.0,86376.421684,0.0,2,2296.166981,6.421684,core 2
13724,Camera,86380.0,86380.0,86386.676891,0.0,2,1796.865973,6.676891,core 4


# 2)
### Compute/display queue statistics for each IoT sensor.


In [6]:
camera_stat = home.compute_waiting_time_for_sensor('Camera')
gas_stat = home.compute_waiting_time_for_sensor('GasDetector')
Intrusion_stat = home.compute_waiting_time_for_sensor('IntrusionDetector')
vocal_stat = home.compute_waiting_time_for_sensor('VocalDetector')
temp_stat = home.compute_waiting_time_for_sensor('TempDetector')
queue_stats = {
    'name':['Camera','Gas','Intrusion','Vocal','Temp'],
    'Queue_stats(sum)':[np.sum(camera_stat),np.sum(gas_stat),np.sum(Intrusion_stat),np.sum(vocal_stat),np.sum(temp_stat)],
    'Queue_stats(mean)':[np.mean(camera_stat),np.mean(gas_stat),np.mean(Intrusion_stat),np.mean(vocal_stat),np.mean(temp_stat)]
   
}


w1= pd.DataFrame(queue_stats)
print(f'Time spend in the queue :')
w1.head()

Time spend in the queue :


,name,Queue_stats(sum),Queue_stats(mean)
0,Camera,4246.577439,0.491559
1,Gas,100.337345,0.069727
2,Intrusion,537.347398,0.186644
3,Vocal,133.671011,0.460935
4,Temp,1776.646385,3.709074


# Comment :
### i noticed that the TempDetector is disadvantaged because he is the only one with the big mean waiting time in the queue

# Solution : 
### I tried many combinations and found out that raising the priority level of TempDetector to High has divided its waiting time by 2 

In [7]:
class HomeIot():
    def __init__(self,cores=['core 1','core 2','core 3','core 4'],capacity=128*10**6):
        self.nb_rejected=0
        self.queue = []
        self.cores_usage={
            'core 1':0,
            'core 2':0,
            'core 3':0,
            'core 4':0
        }
        self.capacity = capacity
        self.cores =cores
        self.sensors=[]
        self.data={'name':[],
                   'arrival_time':[],
                   'begin_time':[],
                   'ending_time':[],
                   'waiting_time':[],
                   'priority':[],
                   'data_size(Ko)':[],
                   'process_time':[],
                   'cores':[],
                   
                    }

        self.dataset= {
            'nb_arrivals': 0,
            'nb_rejected': 0,
            'sum_process_time':0,
            'waiting_time':[]
        }
        self.env = simpy.Environment()
        self.ressources = simpy.Resource(self.env,len(self.cores))
        sensor1= sensor('IntrusionDetector',30,st.norm(128*(10**3),32*(10**3)),4,15,fixed_size=False)
        self.generator = self.env.process(self.sensors_arrival(sensor1))

        sensor2= sensor('GasDetector',60,10**3,4,5)
        self.generator = self.env.process(self.sensors_arrival(sensor2))

        sensor3= sensor('VocalDetector',st.expon(0,3600/12),256*10**3,3,10,fixed_arrival=False)
        self.generator = self.env.process(self.sensors_arrival(sensor3))

        sensor4= sensor('Camera',10,st.norm(2*(10**6),512*(10**3)),2,15,fixed_size=False)
        self.generator = self.env.process(self.sensors_arrival(sensor4))

        sensor5= sensor('TempDetector',180,10**3,3,5)#Increase level to High(3)
        self.generator = self.env.process(self.sensors_arrival(sensor5))
        

    def compute_waiting_time_for_sensor(self, sensor_name):
        data = pd.DataFrame(self.data)
        sensor_data = data[data['name'] == sensor_name]
        
        return list(sensor_data['waiting_time'])
    
    def sensors_arrival(self,sensor):
        # print('run')    

        while True :

            self.dataset['nb_arrivals'] +=1
            yield self.env.timeout(sensor.get_arrival_rate())
            self.arrival_time = self._clock()
            self.data_size = sensor.get_data_size()
            self.sensor_hold = Sensor_hold(sensor.name,self.arrival_time,self.data_size,sensor.priority_level,sensor.get_processing_time())         
            self.sensors.append(sensor)
            if (self.capacity - self.sensor_hold.data_size) > 0  :
                self.queue.append(self.sensor_hold)

                self.capacity-=self.sensor_hold.data_size
                
            else:
                self.dataset['nb_rejected'] +=1
            self.env.process(self.process_sensors())
        

        

    def process_sensors(self):
        arrival_time = self._clock()
        with self.ressources.request() as req:
            yield req

            if not self.queue:
                # The queue is empty, nothing to process
                return

            sensor_index = self.get_sensor()
            sensor = self.queue.pop(sensor_index)
            begin_time = self._clock()            
            waiting_time = begin_time - sensor.arrival_time
            ending_time = begin_time + sensor.process_time
            self.data['name'].append(sensor.name)
            self.data['arrival_time'].append(sensor.arrival_time)
            self.data['begin_time'].append(begin_time)
            self.data['ending_time'].append(ending_time)
            self.data['waiting_time'].append(waiting_time)
            self.data['priority'].append(sensor.priority_level)
            self.data['data_size(Ko)'].append(sensor.data_size/10**3)
            self.data['process_time'].append(sensor.process_time)

            self.capacity += sensor.data_size
            self.dataset['waiting_time'].append(waiting_time)
            core = self.cores.pop(0)
            core_begin= self._clock()         
            self.data['cores'].append(core) 
            yield self.env.timeout(sensor.process_time)
            self.cores.append(core)
            core_end= self._clock()         
            self.cores_usage[core]+=core_end - core_begin

    

    # Get the instance with the highest priority level
    def get_sensor(self):
        k=0
       
        for index,sensor in enumerate(self.queue):
            if sensor.priority_level> self.queue[k].priority_level:
                k= index
        
        return k
        
    def _clock(self): 
        return self.env.now
    
    def run(self, stop_time=100, print_details=False):   
        self.env.run(until=stop_time)  # launch the simulation for a given duration
        self.stats()
        
    def stats(self):
        df = self.dataset
        df['mean_proc']=df['sum_process_time']/ (df['nb_arrivals']-df['nb_rejected'])
        df['percentage_rejected']=df['nb_rejected']*100/df['nb_arrivals']







In [8]:
home = HomeIot()
stop_time = 24*3600
home.run(stop_time)


In [9]:
camera_stat = home.compute_waiting_time_for_sensor('Camera')
gas_stat = home.compute_waiting_time_for_sensor('GasDetector')
Intrusion_stat = home.compute_waiting_time_for_sensor('IntrusionDetector')
vocal_stat = home.compute_waiting_time_for_sensor('VocalDetector')
temp_stat = home.compute_waiting_time_for_sensor('TempDetector')
queue_stats = {
    'name':['Camera','Gas','Intrusion','Vocal','Temp'],
    'Queue_stats(sum)':[np.sum(camera_stat),np.sum(gas_stat),np.sum(Intrusion_stat),np.sum(vocal_stat),np.sum(temp_stat)],
    'Queue_stats(mean)':[np.mean(camera_stat),np.mean(gas_stat),np.mean(Intrusion_stat),np.mean(vocal_stat),np.mean(temp_stat)]
   
}


w1= pd.DataFrame(queue_stats)
print(f'Time spend in the queue :')
w1.head()

Time spend in the queue :


,name,Queue_stats(sum),Queue_stats(mean)
0,Camera,4842.856823,0.560581
1,Gas,87.138119,0.060555
2,Intrusion,510.856587,0.177442
3,Vocal,128.926544,0.479281
4,Temp,611.499462,1.276617
